<a href="https://colab.research.google.com/github/srikanthrc/running-to-stand-still/blob/master/_notebooks/2020-03-21-covid19_overview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COVID-19 Tracking / Worldwide
> Tracking coronavirus total cases, deaths and new cases by country.  Separately, a view of US by state.

- author: Srikanth Chinmay (inspired by Pratap Vardhan @ http://covid19dashboards.com)
- categories: [covid-19]
- image: images/covid-overview.png
- permalink: /covid-overview/
- toc: false
- comments: false


In [37]:
#hide
print('''
Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/
''')


Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/



In [0]:
#hide
import numpy as np
import pandas as pd
from datetime import datetime
from jinja2 import Template
from IPython.display import HTML

In [0]:
#hide
from pathlib import Path
if not Path('covid_overview.py').exists():
    ! wget https://raw.githubusercontent.com/srikanthrc/covid-19/master/covid_overview.py

In [0]:
#hide
import covid_overview as covid

In [0]:
#hide
COL_REGION = 'Country/Region'   

kpis_info = [
    {'title': 'Asia', 'prefix': 'Asia'},
    {'title': 'Europe', 'prefix': 'EU'},
    {'title': 'North America', 'prefix': 'NA'}]

data = covid.gen_country_data(kpis_info=kpis_info)

In [42]:
#hide
#display(data['table'].head(10))
data['summary']

{'Asia Cases': 309152,
 'Asia Cases (+)': 11761,
 'Asia Deaths': 11318,
 'Asia Deaths (+)': 344,
 'Cases': 1917319,
 'Cases (+)': 70640,
 'Deaths': 119482,
 'Deaths (+)': 5391,
 'EU Cases': 908788,
 'EU Cases (+)': 26279,
 'EU Deaths': 80028,
 'EU Deaths (+)': 3221,
 'NA Cases': 619653,
 'NA Cases (+)': 27580,
 'NA Deaths': 24936,
 'NA Deaths (+)': 1612,
 'since': Timestamp('2020-04-12 00:00:00'),
 'updated': Timestamp('2020-04-13 00:00:00')}

In [43]:
#hide_input
template = Template(covid.get_template(covid.paths['overview']))
dt_cols, LAST_DATE_I = data['dt_cols'], data['dt_last']

html = template.render(
    D=data['summary'], table=data['table'].head(30),
    newcases=data['newcases'].loc[:, dt_cols[LAST_DATE_I-50]:dt_cols[LAST_DATE_I]],
    KPIS_INFO=kpis_info,
    np=np, pd=pd, enumerate=enumerate
   )
HTML(f'<div>{html}</div>')

#####Visualizations by [Pratap Vardhan](https://twitter.com/PratapVardhan)[^1]

[^1]: Sources: ["COVID-19 Data Repository by Johns Hopkins CSSE"](https://systems.jhu.edu/research/public-health/ncov/) [GitHub](https://github.com/CSSEGISandData/COVID-19). [covidtracking.com](https://covidtracking.com/)